In [ ]:
### %load_ext autoreload

#!pip install --upgrade scipy #actualizamos scipy para que cargue el módulo de señales
import os
import numpy as np
import scipy as sc
from scipy import signal
import matplotlib.pyplot as plt

import numpy as np 
import pandas as pd

%matplotlib inline
from scipy.io import loadmat
import matplotlib.pyplot as plt

from IPython.display import display, clear_output



import os

!pip install biosppy
!pip install pyhrv
!pip install neurokit2
!pip install hrv
!pip install heartpy
!pip install neurokit
#ya vendría instalado spectrum

import heartpy
import biosppy
import neurokit2 as nk
import hrv
import pyhrv

In [ ]:
#Obtenemos todas las señales. Apartado de preprocesado.
#Tenemos 550 señales de training y 250 señales de test.
#Se me ocurre a priori, tendría que comprobarlo, hay un algoritmo en MATLAB (rdmat)
#que también lo saca si no funciona algo parecido a esto.
#Iteramos sobre nombres de archivos para iterar después sobre señales
path = '/kaggle/input/physiological-signals-processing-challenge-2021/data_challenge/training/'

lista_nombres = []
for i in range(551): #Añado un 1 para que me imprima las 550, dado que Python empieza por el 0.
    pat_filename = '{}.hea'.format(i)
    if pat_filename == '0.hea': #Este archivo no existe pero Python se lo inventa
        pass
    else:
        lista_nombres.append(pat_filename)
lista_señales = []
for elem in lista_nombres:
    pat = loadmat(os.path.join(path, elem[:-4] + '.mat'))     #el -4 es porque cubre exactamente
    #el nombre del archivo hasta el inicio sin contar el punto
    lista_señales.append(pat)
#Hacemos el resampleo a 125 Hz, que tengo que mirar cómo se hace.
for elem in lista_señales:
    elem = sc.signal.resample(elem['val'], 125)
#¿Hace falta representar las señales?
#Me interesa representar como mucho los últimos 20 segundos de un ejemplo(paciente 73).
t = np.arange(0, 75000)/125
plt.figure()
plt.subplot(221)
plt.plot(t, lista_señales[1]['val'][0,:75000])
plt.title('Derivación II')
plt.xlim(0, 50)
plt.subplot(222)
plt.plot(t, lista_señales[1]['val'][1,:75000])
plt.title('Derivación V')
plt.xlim(0, 50)
plt.subplot(223)
plt.plot(t, lista_señales[1]['val'][2, :75000])
plt.title('PLETH')
plt.xlim(0, 50)
plt.subplot(224)
plt.plot(t, lista_señales[1]['val'][3, :75000])
plt.title('ABP')
plt.xlim(0, 50)
#Vale, ya he entendido esto. El valor 0 en ese índice corresponde a la derivación II, 
#el valor 1 a la 5, el valor 2 a PLETH y el valor 3 a ABP. 
#Me queda por entender el otro valor :75000, supongo que es el número de muestras?¿ Ni idea.

#Nos dan igual las etiquetas, luego veo si las quito. Las etiquetas son II, V, PLETH, ABP:
#para los dos canales de ECG, el canal que mide la saturación oxígeno arterial por cada latido
# (oximetría) y el canal de la presión arterial.

In [ ]:
#Aplicamos filtro pasobajo en 50 Hz para eliminar la tensión de línea de corriente.
#Aplicamos un notch IIR, la mayoría de la literatura utiliza este orden para filtrado ruido 50Hz
b, a= sc.signal.iirnotch(38, fs = 125, Q = 30)
#Aplicamos un filtro pasobanda 0.5-40Hz para quitar el baseline wander.
c = sc.signal.firwin(64, [0.5, 40], pass_zero = False, fs=125, window='hamming')
#De esta forma también nos quitamos el ruido de EMG a priori. Ese filtro se usa 
#constantemente en todos los papers consultados
#Eliminamos valores nulos NaN
lista_filtrada_II = []
lista_filtrada_V = []
lista_filtrada_ABP = []
lista_filtrada_PLETH = []
for i in range (551):
    try:#eliminamos tendencia lineal
        elem1 = sc.signal.filtfilt(b, a, lista_señales[i]['val'][0, :75000]) 
        elem1 = sc.signal.filtfilt(c, 1, elem1)
        elem1[np.isnan(elem1)] = 0.00001
        elem1 = sc.signal.detrend(elem1)#eliminamos tendencia lineal
        elem1 = heartpy.enhance_peaks(elem1, iterations = 2)#mejorar picos
        lista_filtrada_II.append(elem1)
        elem2 = sc.signal.filtfilt(b, a, lista_señales[i]['val'][1, :75000])
        elem2 = sc.signal.filtfilt(c, 1, elem2)
        elem2[np.isnan(elem2)] = 0.00001
        elem2 = sc.signal.detrend(elem2)#eliminamos tendencia lineal
        elem2 = heartpy.enhance_peaks(elem2, iterations = 2)#mejorar picos
        lista_filtrada_V.append(elem2)
        elem3 = sc.signal.filtfilt(b, a, lista_señales[i]['val'][2, :75000])
        elem3 = sc.signal.filtfilt(c,1, elem3)
        elem3[np.isnan(elem3)] = 0.00001
        elem3 = sc.signal.detrend(elem3)#eliminamos tendencia lineal
        elem3 = heartpy.enhance_peaks(elem3, iterations = 2)#mejorar picos 
        lista_filtrada_PLETH.append(elem3)
        elem4 = sc.signal.filtfilt(b, a, lista_señales[i]['val'][3, :75000])
        elem4 = sc.signal.filtfilt(c, 1, elem4)
        elem4[np.isnan(elem4)] = 0.00001
        elem4 = sc.signal.detrend(elem4)#eliminamos tendencia lineal
        elem4 = heartpy.enhance_peaks(elem4, iterations = 2)#mejorar picos y evita clippeo
        elem4 = lista_filtrada_ABP.append(elem4)
    except IndexError:
        continue
    #filtramos las señales. El IndexError aparece porque no siempre hay PLETH o ABP
    #para cada uno de los pacientes, y porque no hay paciente 0. Por ello, simplemente 
    #pasamos de ello y seguimos ejecutando el bucle for.
    #Tras representar las señales, no me queda muy claro
    # que haya cambiado algo. Quizás ya se quitó ese ruido 
    # con anterioridad.

t = np.arange(0, 75000)/125
plt.figure()
plt.title('Señales filtradas')
plt.subplot(221)
plt.plot(t, lista_filtrada_II[1])
plt.title('Derivación II')
plt.xlim(0, 50)
plt.subplot(222)
plt.plot(t, lista_filtrada_V[1])
plt.title('Derivación V')
plt.xlim(0, 50)
plt.subplot(223)
plt.plot(t, lista_filtrada_PLETH[1])
plt.title('PLETH')
plt.xlim(0, 50)
plt.subplot(224)
plt.plot(t, lista_filtrada_ABP[1])
plt.title('ABP')
plt.xlim(0, 50)

In [ ]:
#Aquí empezamos con el bloque de adquisición de características. Las características 
#que vamos a obtener son:
#índice de pureza espectral
#calidad de pleth y abp
#media HR y decrecimiento PLETH Y ABP (OSc-ANFc-W)
#maximo de HR intervals en ECG (AMM)
#---------------------------------------------- Linear Discriminant Analysis/ECG--><3.5 Asistolia
#media y mediana HR PLETH Y ABP (OSc-ANFc-W)
#minimo HR en 5 latidos consecutivos (AMM)
#----------------------------------------------media y mediana > 54bpm/ECG-->>40bpm
#HR máximo averaged en ventanas de 3 segundos (OSc-ANFc-W) PLETH-ABP ---> <90bpm
#HR mínimo averaged en ventanas de 3 segundos (OSc-ANFc-W) PLETH-ABP----> >60bpm
#En ECG igual con SPI (uno averaged y el otro incremento máximo) ---> max(SPI) <0.25 & and (max(SPI) <0.36
#----------------------------------------------------------------------max(SPI increase <0.012 and (max(SPI) <0.36 & max(SPI increase) <0.2
#--------------------------------------------------
#max averaged SPI en ventanas de 3 segundos <0.63

In [ ]:
### Obtenemos los picos R de las señales ECG y los intervalos RR
rpeaks_II = []
properties_II = []
rpeaks_V= []
properties_V = []
rr_interval_V = []
rr_interval_II = []
for i in range(len(lista_filtrada_V)):
    rpeaksV, _ = sc.signal.find_peaks(lista_filtrada_V[i], distance = 150)
    rpeaksV = biosppy.signals.ecg.correct_rpeaks(lista_filtrada_V[i], rpeaks = rpeaksV, sampling_rate = 125, tol = 0.04)
    rr_intervalV = np.diff(rpeaksV)/125 * 1000
    rpeaks_V.append(rpeaksV['rpeaks'])
    rr_interval_V.append(rr_intervalV)
for i in range(len(lista_filtrada_II)):
    rpeaksII, _ = sc.signal.find_peaks(lista_filtrada_II[i], distance = 150)
    rpeaksII = biosppy.signals.ecg.correct_rpeaks(lista_filtrada_II[i], rpeaks = rpeaksII, sampling_rate = 125, tol = 0.04)
    rpeaks_II.append(rpeaksII['rpeaks'])
    rr_intervalII = np.diff(rpeaksII)/125 * 1000
    rr_interval_II.append(rr_intervalII)
for i in range(len(lista_filtrada_V)):
    rpeaksV, propertiesV = sc.signal.find_peaks(lista_filtrada_V[i], distance = 150, prominence = 1, width = 20)
    properties_V.append(propertiesV["width_heights"])
    properties_V.append(propertiesV["prominences"])
for i in range(len(lista_filtrada_II)):
    rpeaksII, propertiesII = sc.signal.find_peaks(lista_filtrada_II[i], distance = 150, prominence = 1, width = 20)
    properties_II.append(propertiesII["width_heights"])
    properties_II.append(propertiesII["prominences"])
#Detección de arritmias, en este caso fibrilación ventricular
#rpeaksV, propertiesV = sc.signal.find_peaks(lista_filtrada_V[40], distance = 150, prominence = 1, width = 20)
#anchura_pico = propertiesV['width_heights'] #te devuelve el ancho del pico
#importante para arritmias por ejemplo donde, al haber más latidos, el pico R es más estrecho
#prominencia = propertiesV['prominences'] #te devuelve cómo de alto es el pico
#en relación con la línea de base.
#Las podemos meter juntas o por separado, ver cómo funciona mejor.

In [ ]:
#Estadísticas principales: media, mediana, amplitud máxima, cuasivarianza, 
#desviación estándar, desviación absoluta, kurtosis, skewness
mediaV = []
medianaV = []
amplitudmaxV = [] 
cuasivaV = []
stdV = []
stdabsV = [] 
kurtV = []
skewV = []
mediaII = [] 
medianaII = []
amplitudmaxII = [] 
cuasivaII = []
stdII = []
stdabsII = [] 
kurtII = []
skewII = []
mediaPLETH = [] 
medianaPLETH = []
amplitudmaxPLETH = [] 
cuasivaPLETH = []
stdPLETH = []
stdabsPLETH = [] 
kurtPLETH = []
skewPLETH = []
for i in range(len(lista_filtrada_V)):
    a = biosppy.signals.tools.signal_stats(lista_filtrada_V[i])
    mediaV.append(a[0])
    medianaV.append(a[1])
    amplitudmaxV.append(a[2])
    cuasivaV.append(a[3])
    stdV.append(a[4])
    stdabsV.append(a[5])
    kurtV.append(a[6])
    skewV.append(a[7])
for i in range(len(lista_filtrada_II)):
    b = biosppy.signals.tools.signal_stats(lista_filtrada_II[i])
    mediaII.append(b[0])
    medianaII.append(b[1])
    amplitudmaxII.append(b[2])
    cuasivaII.append(b[3])
    stdII.append(b[4])
    stdabsII.append(b[5])
    kurtII.append(b[6])
    skewII.append(b[7])
for i in range(len(lista_filtrada_PLETH)):
    c = biosppy.signals.tools.signal_stats(lista_filtrada_PLETH[i])
    mediaPLETH.append(c[0])
    medianaPLETH.append(c[1])
    amplitudmaxPLETH.append(c[2])
    cuasivaPLETH.append(c[3])
    stdPLETH.append(c[4])
    stdabsPLETH.append(c[5])
    kurtPLETH.append(c[6])
    skewPLETH.append(c[7])


In [ ]:
#Calculamos la autocorrelación
autocorV = []
autocorII = []
autocorPLETH = []
autocorABP = []
for i in range(len(lista_filtrada_V)):
    autocor_V = np.correlate(lista_filtrada_V[i], lista_filtrada_V[i])
    autocorV.append(autocor_V)
for i in range(len(lista_filtrada_II)):
    autocor_II = np.correlate(lista_filtrada_II[i], lista_filtrada_II[i])
    autocorII.append(autocor_II)
for i in range(len(lista_filtrada_PLETH)):
    autocor_PLETH = np.correlate(lista_filtrada_PLETH[i], lista_filtrada_PLETH[i])
    autocorPLETH.append(autocor_PLETH)
for i in range(len(lista_filtrada_ABP)):
    autocor_ABP = np.correlate(lista_filtrada_ABP[i], lista_filtrada_ABP[i])
    autocorABP.append(autocor_ABP)

In [ ]:
#Obtenemos ciertas características del PLETH. 
#De momento, no obtenemos características concretas de ABP.
picos_sistolicos = []
for i in range(len(lista_filtrada_PLETH)):
    try: #Calculamos las muestras donde se encuentran los picos sistólicos
        sistolic_peaks = nk.ppg_findpeaks(lista_filtrada_PLETH[i], sampling_rate = 125)  
        picos_sistolicos.append(sistolic_peaks['PPG_Peaks'][0])
        
    except IndexError: #Probablemente salte por el mismo motivo que antes
        #por no haber suficientes latidos (habrá una bradicardia)
        #Salta en 4, 64, 71, 289, 336 y 434.
        picos_sistolicos.append(0)
#Calculamos la tasa cardiaca en PLETH
heart_rate_PLETH = []
for i in range(len(picos_sistolicos)):
    tasa_cardiaca_ppg = nk.ppg_rate(picos_sistolicos[i], sampling_rate = 125)
    heart_rate_PLETH.append(tasa_cardiaca_ppg)

In [ ]:
#A continuación, nos disponemos a calcular características del propio ECG.
#bpm instantáneo
templates_II = []
beats_II = []
templates_V = []
beats_V = []
for i in range(len(lista_filtrada_II)):
    templates, rpiks = biosppy.signals.ecg.extract_heartbeats(lista_filtrada_II[i], rpeaks_II[i], sampling_rate = 125)
    templates_II.append(templates)
    beats_II.append(rpiks)
for i in range(len(lista_filtrada_V)):
    templates, rpiks = biosppy.signals.ecg.extract_heartbeats(lista_filtrada_V[i], rpeaks_V[i], sampling_rate = 125)
    templates_V.append(templates)
    beats_V.append(rpiks)
heart_rate_inst_II = []
heart_rate_inst_V = []
for i in range(len(beats_II)):
    try:
        index, heart_rate = biosppy.signals.tools.get_heart_rate(beats_II[i], sampling_rate = 125)
        heart_rate_inst_II.append(heart_rate)
    except ValueError: #Salta porque hay bradicardias probablemente
        heart_rate_inst_II.append(50)
for i in range(len(beats_V)):
    try:
        index, heart_rate = biosppy.signals.tools.get_heart_rate(beats_V[i], sampling_rate = 125)
        heart_rate_inst_V.append(heart_rate)
    except ValueError: #Salta porque hay bradicardias probablemente
        heart_rate_inst_II.append(50)

In [ ]:
#Sacamos todos los onsets ,offsets y el resto de picos aparte de R que conforman PQRS
#ecg_delineate()
tpeaks_V = []
toffsets_V = []
ppeaks_V = []
ponsets_V = []
tpeaks_II = []
toffsets_II = []
ppeaks_II = []
ponsets_II = []
for i in range(len(lista_filtrada_V)):
    try:
        proceso = nk.ecg_delineate (lista_filtrada_V[i], rpeaks = rpeaks_V, sampling_rate = 125, method = 'dwt') 
        tpeaks_V.append(proceso[0]['ECG_T_Peaks'])
        toffsets_V.append(proceso[0]['ECG_T_Offsets'])
        ppeaks_V.append(proceso[0]['ECG_P_Peaks'])
        ponsets_V.append(proceso[0]['ECG_P_Onsets'])
    except ValueError: #Taquicardia
        tpeaks_V.append(0.0001)
        toffsets_V.append(0.0001)
        ppeaks_V.append(0.0001)
        ponsets_V.append(0.0001)
    except IndexError:
        tpeaks_V.append(0.0001)
        toffsets_V.append(0.0001)
        ppeaks_V.append(0.0001)
        ponsets_V.append(0.0001)
for i in range(len(lista_filtrada_V)):
    try:
        proceso = nk.ecg_delineate (lista_filtrada_II[i], rpeaks = rpeaks_V, sampling_rate = 125, method = 'dwt') 
        tpeaks_II.append(proceso[0]['ECG_T_Peaks'])
        toffsets_II.append(proceso[0]['ECG_T_Offsets'])
        ppeaks_II.append(proceso[0]['ECG_P_Peaks'])
        ponsets_II.append(proceso[0]['ECG_P_Onsets'])
    except ValueError:#Taquicardias
        tpeaks_II.append(0.0001)
        toffsets_II.append(0.0001)
        ppeaks_II.append(0.0001)
        ponsets_II.append(0.0001)

In [ ]:
#Seguimos obteniendo características del ECG, ahora en relación con la tasa de variación cardiaca

rmssd_V = []
rmssd_II = []
meanNN_V = []
meanNN_II = []
sdNN_V = []
sdNN_II = []
sdsd_V = []
sdsd_II = []
cvNN_V = []
cvNN_II = []
CVSD_V = []
CVSD_II = []
medianNN_V = [] 
medianNN_II = []
madNN_V = []
madNN_II = []
mcvNN_V = []
mcvNN_II = []
pNN50_V = []
pNN50_II = []
pNN20_V = []
pNN20_II = []
for i in range(len(lista_filtrada_V)):
    statistics = nk.hrv_time (rpeaks_V[i], sampling_rate = 125)
    rmssd_V.append(statistics['HRV_RMSSD'][0])
    meanNN_V.append(statistics['HRV_MeanNN'][0])
    sdNN_V.append(statistics['HRV_SDNN'][0])
    sdsd_V.append(statistics['HRV_SDSD'][0])
    cvNN_V.append(statistics['HRV_CVNN'][0])
    CVSD_V.append(statistics['HRV_CVSD'][0])
    medianNN_V.append(statistics['HRV_MedianNN'][0])
    madNN_V.append(statistics['HRV_MadNN'][0])
    mcvNN_V.append(statistics['HRV_MCVNN'][0])
    pNN50_V.append(statistics['HRV_pNN50'][0])
    pNN20_V.append(statistics['HRV_pNN20'][0])
for i in range(len(lista_filtrada_II)):
    statistics = nk.hrv_time (rpeaks_II[i], sampling_rate = 125)
    rmssd_II.append(statistics['HRV_RMSSD'][0])
    meanNN_II.append(statistics['HRV_MeanNN'][0])
    sdNN_II.append(statistics['HRV_SDNN'][0])
    sdsd_II.append(statistics['HRV_SDSD'][0])
    cvNN_II.append(statistics['HRV_CVNN'][0])
    CVSD_II.append(statistics['HRV_CVSD'][0])
    medianNN_II.append(statistics['HRV_MedianNN'][0])
    madNN_II.append(statistics['HRV_MadNN'][0])
    mcvNN_II.append(statistics['HRV_MCVNN'][0])
    pNN50_II.append(statistics['HRV_pNN50'][0])
    pNN20_II.append(statistics['HRV_pNN20'][0])   

In [ ]:
#Calculamos parámetros frecuenciales (potencias en rangos de frecuencia)
lf_V = []
hf_V = []
vhf_V = []
lfn_V = [] #normalizada
hfn_V = [] #normalizada
ratiolfhf_V = []
lf_II = []
hf_II = []
vhf_II = []
lfn_II = [] #normalizada
hfn_II = [] #normalizada
ratiolfhf_II = []
param_frec = nk.hrv_frequency (rpeaks_V[40], sampling_rate = 125)
for i in range(len(lista_filtrada_V)):
    try:
        param_frec = nk.hrv_frequency (rpeaks_V[i], sampling_rate = 125)
        lf_V.append(param_frec['HRV_LF'][0])
        hf_V.append(param_frec['HRV_HF'][0])
        vhf_V.append(param_frec['HRV_VHF'][0])
        ratiolfhf_V.append(param_frec['HRV_LFHF'][0])
        lfn_V.append(param_frec['HRV_LFn'][0])
        hfn_V.append(param_frec['HRV_HFn'][0])
    except IndexError: #Salta para aquellas muestras a las que previamente les habíamos 
         #asignado un 0 152, 162, 164, 322, 324, 384, 387, 389, 498, 508
        lf_V.append(0.06)
        hf_V.append(0.3)
        vhf_V.append(0.45)
        ratiolfhf_V.append(0.2)
        lfn_V.append(0.2)
        hfn_V.append(0.3)
    except ValueError: #No salta para todas las anteriores pero sí para algunas que 
        #cumplen lo ya dicho, la 162
        lf_V.append(0.06)
        hf_V.append(0.3)
        vhf_V.append(0.45)
        ratiolfhf_V.append(0.2)
        lfn_V.append(0.2)
        hfn_V.append(0.3)
for i in range(len(lista_filtrada_II)):
    try:
        param_frec = nk.hrv_frequency (rpeaks_II[i], sampling_rate = 125)
        lf_II.append(param_frec['HRV_LF'][0])
        hf_II.append(param_frec['HRV_HF'][0])
        vhf_II.append(param_frec['HRV_VHF'][0])
        ratiolfhf_II.append(param_frec['HRV_LFHF'][0])
        lfn_II.append(param_frec['HRV_LFn'][0])
        hfn_II.append(param_frec['HRV_HFn'][0]) 
    except IndexError: #Salta para aquellas muestras a las que previamente les habíamos 
         #asignado un 0 152, 389
        lf_II.append(0.06)
        hf_II.append(0.3)
        vhf_II.append(0.45)
        ratiolfhf_II.append(0.2)
        lfn_II.append(0.2)
        hfn_II.append(0.3)

In [ ]:
#Calculamos la calidad de la señal
quality_V = []
quality_II = []
#152, 162, 164, 322, 324, 384, 387, 389, 498, 508
for i in range(len(lista_filtrada_V)):
    try:
        SQI = nk.ecg_quality(lista_filtrada_V[i], rpeaks_V[i], sampling_rate = 125)
        quality_V.append(SQI[0])
    except IndexError:
        quality_V.append(0.7)
    except ZeroDivisionError:
        quality_V.append(0.7)
    except ValueError:
        quality_V.append(0.7)
for i in range(len(lista_filtrada_II)): #152, 389
    try:
        SQI = nk.ecg_quality(lista_filtrada_II[i], rpeaks_II[i], sampling_rate = 125) 
        quality_II.append(SQI[0])
    except IndexError:
        quality_II.append(0.7)
    except ZeroDivisionError:
        quality_II.append(0.7)
    except ValueError:
        quality_II.append(0.7)

In [ ]:
hrv_V = []
hrv_SD1_V = []
hrv_SD2_V = []
hrv_SD1SD2_V = []
hrv_S_V = []
hrv_CSI_V = []
hrv_CVI_V = []
hrv_CSI_Modified_V = []
hrv_GI_V = []
hrv_SI_V = []
hrv_AI_V = []
hrv_PI_V = []
hrv_SD1d_V = []
hrv_SD1a_V = []
hrv_C1d_V = []
hrv_C1a_V = []
hrv_SD2d_V = []
hrv_SD2a_V = []
hrv_C2d_V = []
hrv_C2a_V = []
hrv_SDNNd_V = []
hrv_SDNNa_V = []
hrv_Cd_V = []
hrv_Ca_V = []
hrv_PIP_V = []
hrv_IALS_V = []
hrv_PSS_V = []
hrv_PAS_V = []
hrv_ApEn_V = []
hrv_SampEn_V = []
hrv_II = []
hrv_SD1_II = []
hrv_SD2_II = []
hrv_SD1SD2_II = []
hrv_S_II = []
hrv_CSI_II = []
hrv_CVI_II = []
hrv_CSI_Modified_II = []
hrv_GI_II = []
hrv_SI_II = []
hrv_AI_II = []
hrv_PI_II = []
hrv_SD1d_II = []
hrv_SD1a_II = []
hrv_C1d_II = []
hrv_C1a_II = []
hrv_SD2d_II = []
hrv_SD2a_II = []
hrv_C2d_II = []
hrv_C2a_II = []
hrv_SDNNd_II = []
hrv_SDNNa_II = []
hrv_Cd_II = []
hrv_Ca_II = []
hrv_PIP_II = []
hrv_IALS_II = []
hrv_PSS_II = []
hrv_PAS_II = []
hrv_ApEn_II = []
hrv_SampEn_II = []
for i in range(len(lista_filtrada_V)):
    try:
        hrv_V = nk.hrv_nonlinear(peaks = rpeaks_V[i] ,sampling_rate=125)
        hrv_SD1_V.append(hrv['HRV_SD1'][0])
        hrv_SD2_V.append(hrv['HRV_SD2'][0])
        hrv_SD1SD2_V.append(hrv['HRV_SD1SD2'][0])
        hrv_S_V.append(hrv['HRV_S'][0])
        hrv_CSI_V.append(hrv['HRV_CSI'][0])
        hrv_CVI_V.append(hrv['HRV_CVI'][0])
        hrv_CSI_Modified_V.append(hrv['HRV_CSI_Modified'][0])
        hrv_GI_V.append(hrv['HRV_GI'][0])
        hrv_SI_V.append(hrv['HRV_SI'][0])
        hrv_AI_V.append(hrv['HRV_AI'][0])
        hrv_PI_V.append(hrv['HRV_PI'][0])
        hrv_SD1d_V.append(hrv['HRV_SD1d'][0])
        hrv_SD1a_V.append(hrv['HRV_SD1a'][0])
        hrv_C1d_V .append(hrv['HRV_C1d'][0])
        hrv_C1a_V.append(hrv['HRV_C1a'][0])
        hrv_SD2d_V.append(hrv['HRV_SD2d'][0])
        hrv_SD2a_V.append(hrv['HRV_SD2a'][0])
        hrv_C2d_V.append(hrv['HRV_C2d'][0])
        hrv_C2a_V.append(hrv['HRV_C2a'][0])
        hrv_SDNNd_V.append(hrv['HRV_SDNNd'][0])
        hrv_SDNNa_V.append(hrv['HRV_SDNNa'][0])
        hrv_Cd_V.append(hrv['HRV_Cd'][0])
        hrv_Ca_V.append(hrv['HRV_Ca'][0])
        hrv_PIP_V.append(hrv['HRV_PIP'][0])
        hrv_IALS_V.append(hrv['HRV_IALS'][0])
        hrv_PSS_V.append(hrv['HRV_PSS'][0])
        hrv_PAS_V.append(hrv['HRV_PAS'][0])
        hrv_ApEn_V.append(hrv['HRV_ApEn'][0])
        hrv_SampEn_V.append(hrv['HRV_SampEn'][0])
    except: #ZeroDivisionError, ValueError
        hrv_SD1_V.append(0.01)
        hrv_SD2_V.append(0.01)
        hrv_SD1SD2_V.append(0.01)
        hrv_S_V.append(0.01)
        hrv_CSI_V.append(0.01)
        hrv_CVI_V.append(0.01)
        hrv_CSI_Modified_V.append(0.01)
        hrv_GI_V.append(0.01)
        hrv_SI_V.append(0.01)
        hrv_AI_V.append(0.01)
        hrv_PI_V.append(0.01)
        hrv_SD1d_V.append(0.01)
        hrv_SD1a_V.append(0.01)
        hrv_C1d_V .append(0.01)
        hrv_C1a_V.append(0.01)
        hrv_SD2d_V.append(0.01)
        hrv_SD2a_V.append(0.01)
        hrv_C2d_V.append(0.01)
        hrv_C2a_V.append(0.01)
        hrv_SDNNd_V.append(0.01)
        hrv_SDNNa_V.append(0.01)
        hrv_Cd_V.append(0.01)
        hrv_Ca_V.append(0.01)
        hrv_PIP_V.append(0.01)
        hrv_IALS_V.append(0.01)
        hrv_PSS_V.append(0.01)
        hrv_PAS_V.append(0.01)
        hrv_ApEn_V.append(0.01)
        hrv_SampEn_V.append(0.01)
for i in range(len(lista_filtrada_II)):
    try:
        hrv_II = nk.hrv_nonlinear(peaks = rpeaks_II[i], sampling_rate = 125)
        hrv_SD1_II.append(hrv['HRV_SD1'][0])
        hrv_SD2_II.append(hrv['HRV_SD2'][0])
        hrv_SD1SD2_II.append(hrv['HRV_SD1SD2'][0])
        hrv_S_II.append(hrv['HRV_S'][0])
        hrv_CSI_II.append(hrv['HRV_CSI'][0])
        hrv_CVI_II.append(hrv['HRV_CVI'][0])
        hrv_CSI_Modified_II.append(hrv['HRV_CSI_Modified'][0])
        hrv_GI_II.append(hrv['HRV_GI'][0])
        hrv_SI_II.append(hrv['HRV_SI'][0])
        hrv_AI_II.append(hrv['HRV_AI'][0])
        hrv_PI_II.append(hrv['HRV_PI'][0])
        hrv_SD1d_II.append(hrv['HRV_SD1d'][0])
        hrv_SD1a_II.append(hrv['HRV_SD1a'][0])
        hrv_C1d_II.append(hrv['HRV_C1d'][0])
        hrv_C1a_II.append(hrv['HRV_C1a'][0])
        hrv_SD2d_II.append(hrv['HRV_SD2d'][0])
        hrv_SD2a_II.append(hrv['HRV_SD2a'][0])
        hrv_C2d_II.append(hrv['HRV_C2d'][0])
        hrv_C2a_II.append(hrv['HRV_C2a'][0])
        hrv_SDNNd_II.append(hrv['HRV_SDNNd'][0])
        hrv_SDNNa_II.append(hrv['HRV_SDNNa'][0])
        hrv_Cd_II.append(hrv['HRV_Cd'][0])
        hrv_Ca_II.append(hrv['HRV_Ca'][0])
        hrv_PIP_II.append(hrv['HRV_PIP'][0])
        hrv_IALS_II.append(hrv['HRV_IALS'][0])
        hrv_PSS_II.append(hrv['HRV_PSS'][0])
        hrv_PAS_II.append(hrv['HRV_PAS'][0])
        hrv_ApEn_II.append(hrv['HRV_ApEn'][0])
        hrv_SampEn_II.append(hrv['HRV_SampEn'][0])
    except: #ValueError, ZeroDivisionError
        hrv_SD1_II.append(0.01)
        hrv_SD2_II.append(0.01)
        hrv_SD1SD2_II.append(0.01)
        hrv_S_II.append(0.01)
        hrv_CSI_II.append(0.01)
        hrv_CVI_II.append(0.01)
        hrv_CSI_Modified_II.append(0.01)
        hrv_GI_II.append(0.01)
        hrv_SI_II.append(0.01)
        hrv_AI_II.append(0.01)
        hrv_PI_II.append(0.01)
        hrv_SD1d_II.append(0.01)
        hrv_SD1a_II.append(0.01)
        hrv_C1d_II.append(0.01)
        hrv_C1a_II.append(0.01)
        hrv_SD2d_II.append(0.01)
        hrv_SD2a_II.append(0.01)
        hrv_C2d_II.append(0.01)
        hrv_C2a_II.append(0.01)
        hrv_SDNNd_II.append(0.01)
        hrv_SDNNa_II.append(0.01)
        hrv_Cd_II.append(0.01)
        hrv_Ca_II.append(0.01)
        hrv_PIP_II.append(0.01)
        hrv_IALS_II.append(0.01)
        hrv_PSS_II.append(0.01)
        hrv_PAS_II.append(0.01)
        hrv_ApEn_II.append(0.01)
        hrv_SampEn_II.append(0.01)

In [ ]:
#Calculamos rr intervals maximos y minimos
max_int_RR_II = []
max_int_RR_V = []
for i in range(len(rr_interval_V)):
    elem = max(rr_interval_V[i])
    max_int_RR_V.append(elem)
for i in range(len(rr_interval_II)):
    elem = max(rr_interval_II[i])
    max_int_RR_II.append(elem)
#Calculamos estadisticos del heart rate
media_hrV = []
minimo_hrV = []
maximo_hrV = []
desviacion_hrV = []
media_hrII = []
minimo_hrII = []
maximo_hrII = []
desviacion_hrII = []
for i in range(len(rr_interval_V)):
    try:
        estadisticos_hr = pyhrv.time_domain.hr_parameters(nni = rr_interval_V[i])
        media_hrV.append(estadisticos_hr[0])
        minimo_hrV.append(estadisticos_hr[1])
        maximo_hrV.append(estadisticos_hr[2])
        desviacion_hrV.append(estadisticos_hr[3])
    except: #152, 164, 322, 324, 384, 387, 389, 498, 508, IndexError, ValueError
        media_hrV.append(45)
        minimo_hrV.append(25)
        maximo_hrV.append(51)
        desviacion_hrV.append(2)
for i in range(len(rr_interval_II)):
    try:
        estadisticos_hr = pyhrv.time_domain.hr_parameters(nni = rr_interval_II[i])
        media_hrII.append(estadisticos_hr[0])
        minimo_hrII.append(estadisticos_hr[1])
        maximo_hrII.append(estadisticos_hr[2])
        desviacion_hrII.append(estadisticos_hr[3])
    except ValueError:#153, 389
        media_hrII.append(45)
        minimo_hrII.append(25)
        maximo_hrII.append(51)
        desviacion_hrII.append(2)
#Calculamos diferencias entre intervalos NN (picos R normalizados)
media_nni_dif_II = []
min_nni_dif_II = []
max_nni_dif_II = []
media_nni_dif_V = []
min_nni_dif_V = []
max_nni_dif_V = []
for i in range(len(rr_interval_V)):
    try:
        nni_differences = pyhrv.time_domain.nni_differences_parameters(nni = rr_interval_II[i][0])
        media_nni_dif_V.append(nni_differences['nni_diff_mean'])
        min_nni_dif_V.append(nni_differences['nni_diff_min'])
        max_nni_dif_V.append(nni_differences['nni_diff_max'])
    except IndexError:#152, 164, 322, 324, 384, 387, 389, 498, 508
        media_nni_dif_V.append(282)
        min_nni_dif_V.append(0)
        max_hrV.append(1248)

    except ValueError:#153, 389
        media_nni_dif_V.append(282)
        min_nni_dif_V.append(0)
        max_nni_dif_V.append(1248)

for i in range(len(rr_interval_II)):
    try:
        nni_differences = pyhrv.time_domain.nni_differences_parameters(nni = rr_interval_II[i][0])
        media_nni_dif_II.append(nni_differences['nni_diff_mean'])
        min_nni_dif_II.append(nni_differences['nni_diff_min'])
        max_nni_dif_II.append(nni_differences['nni_diff_max'])
    except ValueError:#153, 389
        media_nni_dif_II.append(282)
        min_nni_dif_II.append(0)
        max_nni_dif_II.append(1248)

In [ ]:
from IPython.display import display, clear_output
index = []
for i in range(len(lista_filtrada_II)):
    index.append(i)
d = {'index': index, 'col1': media_hrII, 'col2': desviacion_hrII}
X = pd.DataFrame(data = d) 
X = np.array(X)
print(X.shape)

In [ ]:
from IPython.display import display, clear_output
index = []
for i in range(len(lista_filtrada_II)):
    index.append(i)
d = {'index': index, 'col1': media_hrII, 'col2': desviacion_hrII, 'col3': hrv_SD1SD2_II, 
    'col4' : hrv_SD1SD2_V, 'col5': media_hrV, 'col6': desviacion_hrV, 'col7': hrv_SampEn_II, 
    'col8': hrv_SampEn_V}
X = pd.DataFrame(data = d) 
X = np.array(X)
print(X.shape)

In [ ]:
#Get labels for the patients
y = np.loadtxt('/kaggle/input/physiological-signals-processing-challenge-2021/alarms_training.csv',skiprows=1,delimiter = ',',usecols = [0,1])

y_train = y[np.array(X[:,0]-1,dtype = int),:]

In [ ]:
#Create naive bayes
from sklearn.naive_bayes import GaussianNB

#get priors

P_h0 = np.mean(y_train[:,1] == 0)
P_h1 = np.mean(y_train[:,1] == 1)

print('P_h0:',P_h0,'; P_h1:',P_h1)

#naive bayes model

nb_detector = GaussianNB(priors = [P_h0,P_h1])

#train the model
nb_detector.fit(X[:,1:],y_train[:,1])

print("mean values n_classes, n_features")
print(nb_detector.theta_)

print("variance values n_classes, n_features")
print(nb_detector.sigma_)

In [ ]:
#predict first case

x_1 = X[0,1:]

D = nb_detector.predict(x_1[np.newaxis,:])

print(X[0,0])
print(y_train[0,0])
print('Detection: %d'%D[0])
print('Hypothesis H: %d' %y_train[0,1])

#whole training set
y_hat = nb_detector.predict(X[:,1:])


print('ACC = %.2f'%np.mean(y_hat == y_train[:,1]))
from sklearn.metrics import f1_score
print('f1 =', f1_score(y_train[:, 1], y_hat))

In [ ]:
#  

X_test = []

for dirname, _, filenames in os.walk('/kaggle/input/physiological-signals-processing-challenge-2021/data_challenge/test/'):
    filenames.sort()
    j = 0
    for filename in filenames:
        #print(filename)
        
        if j%25==0:
            clear_output(wait = True)
            print(j," of ",len(filenames)/2)
            #print(filename)
            
        if 'mat' in filename:
            continue
        #
        j+=1
        #read the pat data
        record = loadmat(os.path.join(dirname,filename[:-4]+'.mat'))
        
        #get the header
        with open(os.path.join(dirname,filename)) as f:
                header = f.readlines()

        #pass to the function to get the mean and std heart rate
        
        mean_hr,std_hr = get_mean_std_hr_one_pat(record,header,plot_flag = False)
        
        X_test.append([int(filename[:-4]),mean_hr,std_hr])

In [ ]:
#sort data
X_test = np.array(X_test)
print(X_test[:3,:])
idx_sort = np.argsort(X_test[:,0])

X_test = X_test[idx_sort,:]

print(X_test[:3,:])

In [ ]:
#predict

y_hat_test = nb_detector.predict(X_test[:,1:])

In [ ]:
print(y_hat_test[:])

#create solution

import pandas as pd

df = pd.DataFrame({'Id': X_test[:,0], 'Category': y_hat_test})

df.to_csv('submission_naive_bayes.csv',index = False)